# 第4章 变形

## 四、问题与练习

In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('data/table.csv')
df.head()

,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
0,S_1,C_1,1101,M,street_1,173,63,34.0,A+
1,S_1,C_1,1102,F,street_2,192,73,32.5,B+
2,S_1,C_1,1103,M,street_2,186,82,87.2,B+
3,S_1,C_1,1104,F,street_2,167,81,80.4,B-
4,S_1,C_1,1105,F,street_4,159,64,84.8,B+


### 1. 问题
#### 【问题一】 上面提到了许多变形函数，如melt/crosstab/pivot/pivot_table/stack/unstack函数，请总结它们各自的使用特点。

- melt：melt函数可以认为是pivot函数的逆操作，将指定的几列压缩成一列。
- crosstab：交叉表。用于两组变量交叉对指定组数据的【聚合】统计，支持表内各维度的normalize
- pivot: 透视表。对df指定列进行"解压缩"。速度快，功能单一，values中要求行列索引唯一
- pivot_table：透视表。pivot升级版，速度较慢，功能丰富。支持多级解压，可以传入函数
- stack：压缩。最基础的变形函数，将横向的指定level索引【整体】搬移到纵向。
- unstack：解压。stack的逆函数，功能上类似于pivot_table。将纵向的指定level索引【整体】搬移到横向。


#### 【问题二】 变形函数和多级索引是什么关系？哪些变形函数会使得索引维数变化？具体如何变化？
- 变形函数与多级索引部分功能类似，都是对原始数据进行多维度的组合分析，对指定数据做一次pivot效果上相当于对df表格局部（values列）增加了一维索引分割。
- stack/unstack 将横向/纵向的指定level索引【整体】搬移到纵向/横向，行列索引总维数不变，一方减少一维，另一方增加一维

#### 【问题三】 请举出一个除了上文提过的关于哑变量方法的例子。（数值类不编码）
pandas.get_dummies(data, prefix=None, prefix_sep='_', dummy_na=False, columns=None, sparse=False, drop_first=False)


In [22]:
df1 = pd.DataFrame([  
            ['1','green' , 'A'],   
            [2,'red'   , 'B'],   
            [3,'blue'  , 'A']])  

df1.columns = ['id','color',  'class'] 
pd.get_dummies(df1,prefix=['id','color',  'class'],prefix_sep='_') 

,id_2,id_3,id_1,color_blue,color_green,color_red,class_A,class_B
0,0,0,1,0,1,0,1,0
1,1,0,0,0,0,1,0,1
2,0,1,0,1,0,0,1,0


#### 【问题四】 使用完stack后立即使用unstack一定能保证变化结果与原始表完全一致吗？
- 不一定，unstack默认将索引还原到最下层，可能需要.swaplevel(axis=1).sort_index(axis=1)等操作才能与原始表一致

#### 【问题五】 透视表中涉及了三个函数，请分别使用它们完成相同的目标（任务自定）并比较哪个速度最快。


In [2]:
#pivot的功能较少
%timeit pd.crosstab(index=df['ID'],columns=df['Gender'],values=df['Height'],aggfunc='min')
%timeit df.pivot(index='ID',columns='Gender',values='Height')
%timeit df.pivot_table(index='ID',columns='Gender',values='Height')

6.18 ms ± 129 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.41 ms ± 13.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
5.86 ms ± 69 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### 【问题六】 既然melt起到了stack的功能，为什么再设计stack函数？
- melt将指定列索引压缩为一列数据
- stack将同一level的列索引转为行索引，功能相当于melt.set_index，而该操作比较常用

### 2. 练习
#### 【练习一】 继续使用上一章的药物数据集：

In [5]:
df_drug = pd.read_csv('data/Drugs.csv',index_col=[1,2,3,0])
df_drug.head()

,,,,DrugReports
State,COUNTY,SubstanceName,YYYY,
VA,ACCOMACK,Propoxyphene,2010,1
OH,ADAMS,Morphine,2010,9
PA,ADAMS,Methadone,2010,2
VA,ALEXANDRIA CITY,Heroin,2010,5
PA,ALLEGHENY,Hydromorphone,2010,5


#### (a) 现在请你将数据表转化成如下形态，每行需要显示每种药物在每个地区的10年至17年的变化情况，且前三列需要排序：
![avatar](picture/drug_pic.png)
#### (b) 现在请将(a)中的结果恢复到原数据表，并通过equal函数检验初始表与新的结果是否一致（返回True）

In [6]:
#(a)
pivoted = df_drug.pivot_table(index = ['State','COUNTY','SubstanceName'],columns='YYYY',values='DrugReports',fill_value='-')\
    .sort_index().reset_index().rename_axis(columns={'YYYY':''})
pivoted.head()

,State,COUNTY,SubstanceName,2010,2011,2012,2013,2014,2015,2016,2017
0,KY,ADAIR,Buprenorphine,-,3,5,4,27,5,7,10
1,KY,ADAIR,Codeine,-,-,1,-,-,-,-,1
2,KY,ADAIR,Fentanyl,-,-,1,-,-,-,-,-
3,KY,ADAIR,Heroin,-,-,1,2,-,1,-,2
4,KY,ADAIR,Hydrocodone,6,9,10,10,9,7,11,3


In [7]:
#(b)转换表格，melt参数：id_vars不需要处理的列，value_vars需要合并的列，value_name合并后的列名
result = pivoted.rename_axis(columns={'':'YYYY'}).melt(id_vars=['State','COUNTY','SubstanceName']\
                    ,value_vars=pivoted.columns[3:],value_name='DrugReports')
print(result.head())
#去‘-’行，并将DrugReports【因加过'-'转为object】格式转回int，然后设置index并按index排序
result = result[result['DrugReports']!='-'].astype({'DrugReports':'int'})\
                    .set_index(list(result.columns[0:4])).sort_index()

  State COUNTY  SubstanceName  YYYY DrugReports
0    KY  ADAIR  Buprenorphine  2010           -
1    KY  ADAIR        Codeine  2010           -
2    KY  ADAIR       Fentanyl  2010           -
3    KY  ADAIR         Heroin  2010           -
4    KY  ADAIR    Hydrocodone  2010           6


In [8]:
#比较equal->True要求：df的顺序、内容、格式、名称等完全一致
result.equals(df_drug.sort_index())#

True

#### 【练习二】 现有一份关于某地区地震情况的数据集，请解决如下问题：

In [9]:
df_eq = pd.read_csv('data/Earthquake.csv')
df_eq.tail()

,日期,时间,维度,经度,方向,距离,深度,烈度
10057,2015.11.18,12:17:48 AM,42.31,42.94,north,81.6,5.0,3.8
10058,1990.01.28,12:22:43 AM,42.70,26.20,north_west,89.5,2.0,0.0
10059,2001.08.09,12:58:14 AM,42.77,26.47,north,90.6,5.0,0.0
10060,1994.06.05,12:20:03 AM,42.41,43.06,north_east,94.3,33.0,0.0
10061,2009.09.09,12:54:13 AM,42.42,43.03,north_east,95.4,5.0,0.0


#### (a) 现在请你将数据表转化成如下形态，将方向列展开，并将距离、深度和烈度三个属性压缩：
![avatar](picture/earthquake_pic.png)


In [10]:
#(a) 压缩距离深度烈度
melt_a = df_eq.melt(id_vars=df_eq.columns[0:5],\
                  value_vars =['距离','深度','烈度'],value_name='数值').rename(columns={'variable':'地震参数'})
melt_a.head()

,日期,时间,维度,经度,方向,地震参数,数值
0,2003.05.20,12:17:44 AM,39.04,40.38,west,距离,0.1
1,2007.08.01,12:03:08 AM,40.79,30.09,west,距离,0.1
2,1978.05.07,12:41:37 AM,38.58,27.61,south_west,距离,0.1
3,1997.03.22,12:31:45 AM,39.47,36.44,south_west,距离,0.1
4,2000.04.02,12:57:38 AM,40.80,30.24,south_west,距离,0.1


In [11]:
# 解压数值部分
indexname = list(df_eq.columns[0:4])
indexname.append('地震参数')
pivoted_a = melt_a.pivot_table(index=indexname,columns='方向',values='数值',fill_value='-').sort_index()

pivoted_a.head(6)

方向                                    east north north_east north_west south  \
日期         时间          维度   经度   地震参数                                          
1912.08.09 12:29:00 AM 40.6 27.2 深度      -     -          -          -     -   
                                 烈度      -     -          -          -     -   
                                 距离      -     -          -          -     -   
1912.08.10 12:23:00 AM 40.6 27.1 深度      -     -          -          -     -   
                                 烈度      -     -          -          -     -   
                                 距离      -     -          -          -     -   

方向                                    south_east south_west west  
日期         时间          维度   经度   地震参数                             
1912.08.09 12:29:00 AM 40.6 27.2 深度           16          -    -  
                                 烈度          6.7          -    -  
                                 距离          4.3          -    -  
1912.08.10 12:23:00 AM 40.6 27.1 深度            -         15    -  
                                 烈度            -          6    -  
                                 距离            -          2    -

In [12]:
pivoted_a.columns[0:4]

Index(['east', 'north', 'north_east', 'north_west'], dtype='object', name='方向')

#### (b) 现在请将(a)中的结果恢复到原数据表，并通过equal函数检验初始表与新的结果是否一致（返回True）

In [13]:
#压缩数值部分
melt_b = pivoted_a.reset_index()
melt_b = melt_b.melt(id_vars =melt_b.columns[0:5], value_vars =melt_b.columns[5:],value_name='数值')
mresult = melt_b[melt_b['数值']!='-'].astype({'数值':'float'}).reindex(columns=melt_a.columns)#调整顺序
# 解压地震参数
display(mresult.head())
pivoted_b = mresult.pivot_table(index = list(df_eq.columns[0:5]), columns='地震参数',values= '数值').reindex(columns=df_eq.columns[5:]).sort_index()
pivoted_b.head()

,日期,时间,维度,经度,方向,地震参数,数值
105,1917.08.08,12:41:10 AM,39.0,27.0,east,深度,15.0
106,1917.08.08,12:41:10 AM,39.0,27.0,east,烈度,4.6
107,1917.08.08,12:41:10 AM,39.0,27.0,east,距离,5.0
162,1921.04.13,12:54:05 AM,38.4,31.8,east,深度,30.0
163,1921.04.13,12:54:05 AM,38.4,31.8,east,烈度,5.1


距离    深度   烈度
日期         时间          维度   经度   方向                        
1912.08.09 12:29:00 AM 40.6 27.2 south_east  4.3  16.0  6.7
1912.08.10 12:23:00 AM 40.6 27.1 south_west  2.0  15.0  6.0
           12:30:00 AM 40.6 27.1 south_west  2.0  15.0  5.2
1912.08.11 12:19:04 AM 40.6 27.2 south_east  4.3  30.0  4.9
           12:20:00 AM 40.6 27.1 south_west  2.0  15.0  4.5

In [14]:
pivoted_b.equals(df_eq.set_index(list(df_eq.columns[0:5])).sort_index())

True